Имеются данные adult.csv (см. в материалах к занятию).  
Целевой переменной является уровень дохода income (крайний правый столбец).  
Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html  

Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.  
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.  

In [1]:
import pandas as pd

In [2]:
adult = pd.read_csv('adult.csv')
adult.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [16]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = adult[ [ 'age', 'workclass', 'education', 'fnlwgt', 'hours-per-week', 'income' ] ]

# столбцы education и workclass являются категориальными переменными
# переведем их в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies( selectedColumns, columns = [ 'workclass','education' ] )

# столбец income является целевой переменной, удаляем его из X
del X['income']
X.head()

,age,fnlwgt,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college
0,25,226802,40,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,38,89814,50,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,28,336951,40,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,44,160323,40,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,18,103497,30,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
# целевая переменная (столбец income) снова является категориальной
# переведем значения столбца в числа, оставив один столбец
from sklearn.preprocessing import LabelEncoder

In [18]:
le = LabelEncoder()
le.fit( adult['income'] )
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [19]:
# записываем в переменную y преобразованный столбец income

y = pd.Series( data = le.transform( adult['income'] ) )
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int64

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [21]:
model = make_pipeline(StandardScaler(),LogisticRegression(max_iter=10000))
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)
model.fit(X_train, y_train)
predictions = model.predict_proba( X_test)
predictions[:5]

array([[0.75068138, 0.24931862],
       [0.92654687, 0.07345313],
       [0.56614895, 0.43385105],
       [0.89633221, 0.10366779],
       [0.68735945, 0.31264055]])

In [22]:
zip(predictions[:,1],y)
model.predict(X_test)
model.score(X_train, y_train)

0.793091345169499

In [23]:
model.score(X_test, y_test)

0.7958779772060329

Качество нашей модели логистической регрессии - 79,5%, что неплохо

ТЕперь через метод опорных векторов

In [24]:
from sklearn.svm import SVC

In [25]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [26]:
clf.predict(X_test)
clf.score(X_train, y_train)

0.80180759893533

In [27]:
clf.score(X_test, y_test)

0.8016788370982052

Метод SVC показал результат немного лучше, чем Логистическая Регрессия - 80%